<a href="https://colab.research.google.com/github/heesukjang/W266_NLP_With_DeepLearning/blob/main/lesson_4_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 4 Notebook: BERT Endeavors

**Description:** After some setup for our standard IMDB movie classification task we will explore BERT (obtained from the [Huggingface Transformer library](https://huggingface.co/docs/transformers/index)) and apply it to text classification (in one way). 

<a id = 'returnToTop'></a>

## Notebook Contents
  * 1. [Setup](#setup) 
  * 2. [Data Acquisition](#dataAcquisition)  
  * 3. [BERT Basics](#bertBasics)
    * 3.1 [Tokenization](#tokenization)
    * 3.2 [Model Structure & Output](#modelOutput)
    * 3.3 [Context Based Embeddings with BERT](#contextualEmbeddings)
  * 4. [Text Classification with BERT (using the Pooler Output)](#BERTClassification)
    * 4.1 [Class Exercise](#classExercise)

  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2023-spring-main/blob/master/materials/lesson_notebooks/lesson_4_BERT.ipynb)

[Return to Top](#returnToTop)  
<a id = 'setup'></a>

## 1. Setup

This notebook requires the tensorflow dataset and other prerequisites that you must download and then store locally. 

In [ ]:
!pip install tensorflow-datasets --quiet

pydot is also helpful, along with **graphviz**.

In [ ]:
!pip install pydot --quiet

For BERT and other Transformer libraries we generally use Huggingface's implementations:

In [ ]:
!pip install transformers --quiet

Ready to do the imports.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds



import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

For the Transformer library we need to import the **tokenizer** and the TensorFlow **model**:

In [ ]:
from transformers import BertTokenizer, TFBertModel

We then continue for now as we have before.

Below is a helper function to plot histories.

In [ ]:
# 4-window plot of loss and accuracy for two models, for comparison

def make_plot(axs,
              model_history1, 
              model_history2, 
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[metric])
            ax1.plot(model_history.history['val_%s' % metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

A small function calculating the cosine similarity may also come in handy:

In [ ]:
def cosine_similarities(vecs):
    for v_1 in vecs:
        similarities = ''
        for v_2 in vecs:
            similarities += ('\t' + str(np.dot(v_1, v_2)/np.sqrt(np.dot(v_1, v_1) * np.dot(v_2, v_2)))[:4])
        print(similarities)

[Return to Top](#returnToTop)  
<a id = 'dataAcquisition'></a>

## 2. Data Acquisition

We will use the IMDB dataset delivered as part of the tensorflow-datasets library, and split into training and test sets. For expedience, we will limit ourselves in terms of train and test examples.

In [ ]:
train_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:80%]', 'test[80%:]'),
    as_supervised=True)

In [ ]:
train_examples, train_labels = next(iter(train_data.batch(20000)))
test_examples, test_labels = next(iter(test_data.batch(5000)))

In [ ]:
train_examples[:4]

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [ ]:
train_labels[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 1])>

[Return to Top](#returnToTop)  
<a id = 'bertBasics'></a>
## 3. BERT Basics

We now need to settle on the pre-trained BERT model we want to use. We will leverage **'bert-base-cased'**.

We need to create the corresponding model and tokenizer:

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


[Return to Top](#returnToTop)  
<a id = 'tokenization'></a>

### 3.1 Tokenization

Tokenization with BERT is interesting. To minimize the number of unknown words, BERT (like most pre-trained transformer models) uses a **subword** model for tokenization. We will see what that means in a second.

Let's start with something simple:

In [ ]:
bert_tokenizer.tokenize('This is great!')

['This', 'is', 'great', '!']

Ok, that is as expected. What about:

In [ ]:
bert_tokenizer.tokenize('This tree is 1253 years old.')

or

In [1]:
bert_tokenizer.tokenize('Pneumonia can be very serious.')
## means based on regular expression

NameError: ignored

Ouch! Many more complex terms are not in BERT's vocabulary and are split up.

**Question:** in what type of NLP problems can this lead to complications?

Next, how do we generate the BERT input with its tokenizer? Fortunately, by now Huggingface's tokenizer implementation makes this rather straightforward:

In [ ]:
bert_tokenizer(['This is great!'])

{'input_ids': [[101, 1188, 1110, 1632, 106, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

To make sure we do this correctly though we may want to specify that we want to have the inputs for TensorFlow (vs. PyTorch), and we may want to do some padding:

In [ ]:
bert_input = bert_tokenizer(['This is great!', 'This is terrible!'], 
              max_length=10,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

bert_input

{'input_ids': <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
array([[ 101, 1188, 1110, 1632,  106,  102,    0,    0,    0,    0],
       [ 101, 1188, 1110, 6434,  106,  102,    0,    0,    0,    0]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], dtype=int32)>}

What do we notice? Look at shapes and values. Does everything make sense?

**Question**: What is the input_id of the CLS token? What is the input_id of the SEP token?

[Return to Top](#returnToTop)  
<a id = 'modelOutput'></a>

### 3.2 Model Structure & Output

Where we have familiarized ourselves with the tokenization, we can now turn to the model and its output. How does that work? Simple!

In [ ]:
len(bert_model.weights)

199

Let's look at the first 3 of these:

In [ ]:
bert_model.weights[0]

<tf.Variable 'tf_bert_model/bert/embeddings/word_embeddings/weight:0' shape=(28996, 768) dtype=float32, numpy=
array([[-0.00054784, -0.04156886,  0.01308366, ..., -0.0038919 ,
        -0.0335485 ,  0.0149841 ],
       [ 0.01688265, -0.03106827,  0.0042053 , ..., -0.01474032,
        -0.03561099, -0.0036223 ],
       [-0.00057234, -0.02673604,  0.00803954, ..., -0.01002474,
        -0.0331164 , -0.01651673],
       ...,
       [-0.00643814,  0.01658491, -0.02035619, ..., -0.04178825,
        -0.049201  ,  0.00416085],
       [-0.00483562, -0.00267701, -0.02901638, ..., -0.05116647,
         0.00449265, -0.01177113],
       [ 0.03134822, -0.02974372, -0.02302896, ..., -0.01454749,
        -0.05249038,  0.02843569]], dtype=float32)>

In [ ]:
bert_model.weights[1]

<tf.Variable 'tf_bert_model/bert/embeddings/token_type_embeddings/embeddings:0' shape=(2, 768) dtype=float32, numpy=
array([[-3.6974233e-03,  1.7510093e-03, -1.4998456e-05, ...,
         4.1503753e-03, -4.3169265e-03,  2.5677346e-04],
       [-2.4836401e-03, -3.9493470e-03, -3.2805414e-03, ...,
        -2.4496014e-03, -2.2277145e-03, -2.3139343e-03]], dtype=float32)>

In [ ]:
bert_model.weights[2]

<tf.Variable 'tf_bert_model/bert/embeddings/position_embeddings/embeddings:0' shape=(512, 768) dtype=float32, numpy=
array([[ 0.02344714,  0.00516326, -0.013507  , ...,  0.00153387,
         0.01396566,  0.00978599],
       [-0.02690891, -0.00475334, -0.00550656, ...,  0.00586852,
         0.00464536, -0.00547428],
       [-0.01394941, -0.00832889,  0.00383158, ...,  0.00617752,
         0.00074595, -0.01051977],
       ...,
       [ 0.0049373 ,  0.01553355,  0.02107639, ..., -0.00957435,
         0.02251508,  0.01383893],
       [ 0.01732728, -0.02141069, -0.00283472, ..., -0.02397991,
        -0.00158155, -0.01289534],
       [ 0.00196406, -0.01695957,  0.00028094, ...,  0.00183421,
        -0.01463418, -0.00022347]], dtype=float32)>

What are those three?  What can you infer from the shape?

Let's look at two more examples.  

In [ ]:
bert_model.weights[5]

<tf.Variable 'tf_bert_model/bert/encoder/layer_._0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32, numpy=
array([[-3.3502415e-02,  2.4027612e-02, -2.0564368e-02, ...,
        -1.3056799e-03, -3.9361402e-02, -2.2783404e-02],
       [ 4.0312465e-02, -2.5828961e-02,  8.4446638e-06, ...,
        -1.3387711e-02, -3.1533711e-02,  3.4254842e-02],
       [-1.8537968e-02,  7.4056715e-02, -1.8200211e-02, ...,
         4.5182254e-02, -3.1906586e-02,  5.0266716e-03],
       ...,
       [-5.5251766e-02,  4.2552739e-02,  1.3469980e-02, ...,
         6.7677222e-02,  1.5837654e-02,  3.7086751e-02],
       [-2.2093264e-02,  6.9746509e-02, -5.0525931e-03, ...,
         1.9251993e-02,  4.2907264e-02, -5.9672974e-02],
       [ 1.9098628e-02, -1.2032607e-02,  2.8985986e-02, ...,
         1.0258724e-02, -8.2653269e-02,  8.1220670e-03]], dtype=float32)>

In [ ]:
bert_model.weights[7]

<tf.Variable 'tf_bert_model/bert/encoder/layer_._0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32, numpy=
array([[ 2.8657176e-02,  4.8005264e-03,  4.9075916e-02, ...,
        -8.8107968e-03,  5.6885206e-03,  5.4868605e-02],
       [-3.0682154e-02, -3.0306482e-03, -1.2872593e-02, ...,
        -3.2314227e-03,  5.2937653e-02, -3.6515070e-03],
       [-2.6920761e-03,  2.4164479e-02, -1.3647523e-02, ...,
         1.1721017e-02,  9.4898082e-03,  1.5124909e-02],
       ...,
       [ 2.3737766e-02,  7.4703549e-03,  3.8690038e-02, ...,
        -2.8807104e-02,  5.6113109e-02, -2.0291412e-02],
       [-7.9208333e-03,  4.4420104e-02,  1.6604092e-02, ...,
         4.6373899e-03,  2.6797490e-02, -2.8913473e-03],
       [-1.0335554e-02,  2.5628829e-02,  7.9800091e-05, ...,
         3.5981752e-02,  4.9961805e-02,  1.4802449e-02]], dtype=float32)>

What are those?

And let's make sure that we look at more layers in class...


Now let's turn to BERT's output.

In [ ]:
bert_output = bert_model(bert_input)
bert_output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 10, 768), dtype=float32, numpy=
array([[[ 0.31789988,  0.3524657 ,  0.15059783, ..., -0.17058721,
          0.35169637,  0.01252164],
        [ 0.29476976, -0.09967348,  0.63946354, ..., -0.03713119,
          0.21884589,  0.35178164],
        [ 0.15745698,  0.6033653 ,  0.71432567, ...,  0.08226287,
          0.301064  ,  0.637356  ],
        ...,
        [-0.08408105,  0.21920003,  0.30455422, ..., -0.08951646,
          0.32351887,  0.2654752 ],
        [ 0.24848077, -0.03157448,  0.32583135, ..., -0.10880928,
          0.48493972,  0.05661324],
        [ 0.22492452, -0.07156646,  0.20357624, ...,  0.04799665,
          0.44845235,  0.1516393 ]],

       [[ 0.33174527,  0.38341573,  0.08286671, ..., -0.22092679,
          0.24292806, -0.13691907],
        [ 0.32388818, -0.0473342 ,  0.6686549 , ..., -0.00434787,
          0.34234896,  0.3566634 ],
        [ 0.2499516 ,  0.7461111 ,  0.35974258, ..

Let's analyze this a bit:

In [ ]:
print('Shape of first BERT output: ', bert_output[0].shape)
print('Shape of second BERT output: ', bert_output[1].shape)

Shape of first BERT output:  (2, 10, 768)
Shape of second BERT output:  (2, 768)


What does that mean? Are the dimensions correct? Why are there 2 outputs? Let's discuss in class. You can (and should!) also go to https://huggingface.co/docs/transformers/model_doc/bert#transformers.TFBertModel and read the documentation. **REALLY(!)**
 critical.

[Return to Top](#returnToTop)  
<a id = 'contextualEmbeddings'></a>

### 3.3 Context-based Embeddings with BERT

Let's look at the word "bank" in a few contexts:

In [ ]:
bert_bank_inputs = bert_tokenizer(["I need to bring my money to the bank today",
                                  "I will need to bring my money to the bank tomorrow",
                                  "I had to bank into a turn",
                                  "The bank teller was very nice" ],
                                padding=True,
                                return_tensors='tf')

bert_bank_inputs

{'input_ids': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
array([[ 101,  146, 1444, 1106, 2498, 1139, 1948, 1106, 1103, 3085, 2052,
         102,    0],
       [ 101,  146, 1209, 1444, 1106, 2498, 1139, 1948, 1106, 1103, 3085,
        4911,  102],
       [ 101,  146, 1125, 1106, 3085, 1154,  170, 1885,  102,    0,    0,
           0,    0],
       [ 101, 1109, 3085, 1587, 1200, 1108, 1304, 3505,  102,    0,    0,
           0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], dtype=int32)

Next, we will get the outputs and extract the word vectors for bank in each of these sentences:

In [ ]:
bert_bank_outputs = bert_model(bert_bank_inputs)

bank_1 = bert_bank_outputs[0][0, 9]
bank_2 = bert_bank_outputs[0][1, 10]
bank_3 = bert_bank_outputs[0][2, 4]
bank_4 = bert_bank_outputs[0][3, 2]

banks = [bank_1, bank_2, bank_3, bank_4]

Where are those numbers coming from?

Finally, we obtain the cosine similarities between the 4 vectors (from left to right and top to bottom we iterate through our vectors and report the cosine similarity):

In [ ]:
cosine_similarities(banks)

	1.0	0.99	0.59	0.86
	0.99	1.0	0.59	0.87
	0.59	0.59	1.0	0.62
	0.86	0.87	0.62	1.0


Does this look right?

[Return to Top](#returnToTop)  
<a id = 'BERTClassification'></a>

# 4. Text Classification with BERT (using the Pooler Output)

The BERT model returns two values that can be exploited for classification purposes. One is the last_hidden_state which is the sequence of hidden-states at the output of the last layer of the model.  The second one is the pooler output, with is the output of the [CLS] token where another linear layer is added on top followed by a tanh. This pooler output can be used for classification purposes.

Let us create the data. More will be discussed in class. (We can limit the training and test data sizes for expedience in class.)

In [ ]:
# BERT Tokenization of training and test data

num_train_examples = 20000      # set number of train examples - 1500 for realtime demo
num_test_examples = 5000        # set number of test examples - 500 for realtime demo

MAX_SEQUENCE_LENGTH = 128                 # set max_length of the input sequence

all_train_examples = [x.decode('utf-8') for x in train_examples.numpy()]
all_test_examples = [x.decode('utf-8') for x in test_examples.numpy()]

x_train = bert_tokenizer(all_train_examples[:num_train_examples],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_train = train_labels[:num_train_examples]

x_test = bert_tokenizer(all_test_examples[:num_test_examples],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_test = test_labels[:num_test_examples]

Now we define the model...

In [ ]:
def create_bert_classification_model(bert_model,
                                     num_train_layers=0,
                                     hidden_size = 200, 
                                     dropout=0.3,
                                     learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes
    """
    if num_train_layers == 0:
        # Freeze all layers of pre-trained BERT model
        bert_model.trainable = False

    elif num_train_layers == 12: 
        # Train all layers of the BERT model
        bert_model.trainable = True

    else:
        # Restrict training to the num_train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(num_train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
          
        
        print('retrain layers: ', retrain_layers)

        for w in bert_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                #print('freezing: ', w)
                w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    bert_out = bert_model(bert_inputs)

    pooler_token = bert_out[1]   # pooler token layer
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')
    
    return classification_model

In [ ]:
bert_classification_model = create_bert_classification_model(bert_model, num_train_layers=0)

In [ ]:
#confirm all layers are frozen
bert_classification_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                              

In [ ]:
keras.utils.plot_model(bert_classification_model, show_shapes=True, dpi=90)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


... and train it!  (This takes a few minutes; we might only be able to train for one epoch in the live session.)

In [ ]:
bert_classification_model_history = bert_classification_model.fit(
    [x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
    y_train,
    validation_data=([x_test.input_ids, x_test.token_type_ids, x_test.attention_mask], y_test),
    batch_size=32,
    epochs=2
)  

Epoch 1/2
119/625 [====>.........................] - ETA: 31:06 - loss: 0.7196 - accuracy: 0.5239

How well does it work? Can we do better? In the code above, we trained the extra hidden and classification layers that we added on top of BERT for our task. But we froze the BERT model (set trainable=False) so we're leaving the pre-trained BERT layers as-is and only our dense layer is learning.
 
[Return to Top](#returnToTop)  
<a id = 'classExercise'></a>

### 4.1 Class Exercise

In this notebook we're using a transformed version of the CLS token (called the pooler output) from the BERT model, which was pre-trained using the next sentence prediction task. It might not capture the useful context from the sentence that we need for our task. So we might need to unfreeze some of the pre-trained BERT layers and update their weights (fine-tune them) for our task.

We'll use the parameter "num_train_layers" in the model building function above. What if we just set num_train_layers=4, to unfreeze only the top four layers of the BERT model? What if we set num_train_layers=12 to unfreeze all of the BERT transformer layers? Does some fine-tuning help? Does too much fine-tuning overfit?

In [ ]:
bert_classification_model = create_bert_classification_model(bert_model, num_train_layers=4)

bert_classification_model_history = bert_classification_model.fit(
    [x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
    y_train,
    validation_data=([x_test.input_ids, x_test.token_type_ids, x_test.attention_mask], y_test),
    batch_size=32,
    epochs=2
)

In [ ]:
bert_classification_model = create_bert_classification_model(bert_model, num_train_layers=12)

In [ ]:
bert_classification_model.summary()

In [ ]:
bert_classification_model_history = bert_classification_model.fit(
    [x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
    y_train,
    validation_data=([x_test.input_ids, x_test.token_type_ids, x_test.attention_mask], y_test),
    batch_size=32,
    epochs=2
)

And that is one way to do text classification with BERT. There are multiple ways (see Assignment 2.)